In [2]:
import pandas as pd
import geopandas as gpd
import os

In [3]:
# List all files in ./data/processed 
files = os.listdir('./data/processed')

# filter files with 'grid' in the name
grid_files = [f for f in files if 'grid' in f]

footprints_files = [f for f in files if 'aggregated' in f]


In [44]:
# Each file in those lists has exactly the same dataset, in the same order of rows, but varying values in the column "value" and "timestamp"
# We will merge all those files into a single file, with the same structure, but with the max value of "value" and the corresponding value of "timestamp"

grid_datasets = [gpd.read_file('./data/processed/' + f) for f in grid_files]
footprints_datasets = [gpd.read_file('./data/processed/' + f) for f in footprints_files]

def merge_by_max_value(frames:list[pd.DataFrame]):
    # Concatenate the GeoDataFrame, including 'geometry' only from the first one
    concat = pd.concat(
        [frames[0][['geometry', 'timestamp', 'value']]] +
        [df[['timestamp', 'value']] for df in frames[1:]],
        axis=1
    )

    # Generate new column names for 'timestamp' and 'value' to avoid overlap
    new_col_names = []
    for i, df in enumerate(frames):
        new_col_names.extend([f'timestamp_{i}', f'value_{i}'])
    concat.columns = ['geometry'] + new_col_names

    # Calculate the maximum 'value' across all files for each row
    max_values = concat.filter(like='value_').max(axis=1)

    # Find the index (position) of the maximum 'value' to determine the corresponding 'timestamp'
    max_value_positions = concat.filter(like='value_').idxmax(axis=1)
    concat['max_value_positions'] = max_value_positions.apply(lambda x: x.replace('value', 'timestamp'))
    concat['max_value_timestamp'] = concat.apply(lambda row: row[row['max_value_positions']], axis=1)
    # Create the final GeoDataFrame
    grid_final = concat[['geometry']]
    grid_final['value'] = max_values
    grid_final['timestamp'] = concat['max_value_timestamp']
    grid_final = gpd.GeoDataFrame(grid_final, geometry='geometry')
    return grid_final

merge_by_max_value(grid_datasets).to_file('./data/grid_final.geojson', driver='GeoJSON')
merge_by_max_value(footprints_datasets).to_file('./data/footprints_final.geojson', driver='GeoJSON')

/Users/hermanmitish/private/satellite-analysis/.conda/lib/python3.11/site-packages/geopandas/geodataframe.py:1525: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
/Users/hermanmitish/private/satellite-analysis/.conda/lib/python3.11/site-packages/geopandas/geodataframe.py:1525: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
/Users/hermanmitish/private/satellite-analysis/.conda/lib/python3.11/site-packages/geopandas/geodataframe.py:1525: SettingWith